In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import model_selection, preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from imblearn.pipeline import make_pipeline
from imblearn.combine import SMOTEENN
from sklearn.feature_selection import SelectFromModel
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier

In [2]:
df=pd.read_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\train_Df64byy.csv')

In [3]:
df1=df.copy()

In [4]:
df.isnull().sum()

ID                             0
City_Code                      0
Region_Code                    0
Accomodation_Type              0
Reco_Insurance_Type            0
Upper_Age                      0
Lower_Age                      0
Is_Spouse                      0
Health Indicator           11691
Holding_Policy_Duration    20251
Holding_Policy_Type        20251
Reco_Policy_Cat                0
Reco_Policy_Premium            0
Response                       0
dtype: int64

In [5]:
df.T.apply(lambda x: x.nunique(), axis=1)

ID                         50882
City_Code                     36
Region_Code                 5316
Accomodation_Type              2
Reco_Insurance_Type            2
Upper_Age                     58
Lower_Age                     60
Is_Spouse                      2
Health Indicator               9
Holding_Policy_Duration       15
Holding_Policy_Type            4
Reco_Policy_Cat               22
Reco_Policy_Premium         6977
Response                       2
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 14 columns):
ID                         50882 non-null int64
City_Code                  50882 non-null object
Region_Code                50882 non-null int64
Accomodation_Type          50882 non-null object
Reco_Insurance_Type        50882 non-null object
Upper_Age                  50882 non-null int64
Lower_Age                  50882 non-null int64
Is_Spouse                  50882 non-null object
Health Indicator           39191 non-null object
Holding_Policy_Duration    30631 non-null object
Holding_Policy_Type        30631 non-null float64
Reco_Policy_Cat            50882 non-null int64
Reco_Policy_Premium        50882 non-null float64
Response                   50882 non-null int64
dtypes: float64(2), int64(6), object(6)
memory usage: 5.4+ MB


In [7]:
df['Holding_Policy_Duration']=df['Holding_Policy_Duration'].replace('14+',15)

In [8]:
df=df.astype({"Holding_Policy_Type":'category', "Reco_Policy_Cat":'category',"City_Code":'category',"Accomodation_Type":'category',"Reco_Insurance_Type":'category',"Holding_Policy_Duration":'category',"Is_Spouse":'category',"Health Indicator":'category','Region_Code':'category'})

In [9]:
#No missing values in num variables
num_vars=df.columns[df.dtypes!='category']
cat_vars=df.columns[df.dtypes=='category']

# MISSING VALUE ANALYSIS & IMPUTATION

In [10]:
df['Health Indicator']=df['Health Indicator'].cat.codes
df['Health Indicator']=df['Health Indicator'].replace(-1, np.nan)
df['Accomodation_Type']=df['Accomodation_Type'].cat.codes
df['Reco_Insurance_Type']=df['Reco_Insurance_Type'].cat.codes
df['Is_Spouse']=df['Is_Spouse'].cat.codes
df['City_Code']=df['City_Code'].cat.codes

# df1['Health Indicator'].isnull().sum()

In [11]:
df2=df.copy()

In [12]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
data_with_imputed_values = pd.DataFrame(my_imputer.fit_transform(df2))

In [13]:
data_with_imputed_values.columns=df2.columns
data_with_imputed_values.index=df2.index

# OUTLIER ANALYSIS

In [14]:
def remove_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.75)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.loc[(df_in[col_name] > fence_low) & (df_in[col_name] < fence_high)]
    return df_out

In [ ]:
df2.shape

In [15]:
data_with_imputed_values=remove_outlier(data_with_imputed_values,'Reco_Policy_Premium')

In [16]:
data_with_imputed_values.shape

(50061, 14)

# standardization

In [17]:
data_with_imputed_values[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']]=StandardScaler().fit_transform(data_with_imputed_values[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']])

# Splitting of Dataset

In [50]:
d1=data_with_imputed_values[:500]
data_with_imputed_values=data_with_imputed_values[500:]

In [210]:
X=data_with_imputed_values.loc[:, data_with_imputed_values.columns != 'ID']
y=data_with_imputed_values['Response']
X.drop(columns=['Response'],inplace=True)

In [248]:
X=X[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age','Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat','Reco_Policy_Premium','Holding_Policy_Type','Reco_Insurance_Type']]

In [256]:
X=X[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Cat','Reco_Policy_Premium']]

In [257]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.001, random_state=100)

In [258]:
len(X_train.columns)

6

In [259]:
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(y_train)
test_y = encoder.fit_transform(y_test)

In [260]:
smote = SMOTE(k_neighbors=5, n_jobs=-1,random_state=777)
smote_enn = make_pipeline(SimpleImputer(), SMOTEENN(smote=smote))
sm_xtrain, sm_ytrain = smote_enn.fit_resample(X, y)

# RANDOM FOREST

In [176]:
from sklearn.ensemble import RandomForestClassifier
rndmf = SelectFromModel(RandomForestClassifier(n_estimators=500,class_weight='balanced'))
rndmf.fit(sm_xtrain, sm_ytrain)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight='balanced',
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=500, n_jobs=None,
                                                 oob_score=

In [177]:
selected_feat= X_train.columns[(rndmf.get_support())]
print(selected_feat)

Index(['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age', 'Reco_Policy_Cat',
       'Reco_Policy_Premium'],
      dtype='object')


In [178]:
# Results from Grid Search 

#  The best estimator across ALL searched params:
#  RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
#                        criterion='gini', max_depth=30, max_features='auto',
#                        max_leaf_nodes=None, max_samples=None,
#                        min_impurity_decrease=0.0, min_impurity_split=None,
#                        min_samples_leaf=1, min_samples_split=2,
#                        min_weight_fraction_leaf=0.0, n_estimators=500,
#                        n_jobs=None, oob_score=False, random_state=None,
#                        verbose=0, warm_start=False)

#  The best score across ALL searched params:
#  0.837041945489787

#  The best parameters across ALL searched params:
#  {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 500}

In [261]:
RF_model= RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight='balanced',
                       criterion='gini', max_depth=30, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [262]:
RF_model=RF_model.fit(sm_xtrain, sm_ytrain)
y_pred1=RF_model.predict(X_test)

In [254]:
y_pred1_proba=RF_model.predict_proba(X_test)

In [263]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred1)

0.787941787941788

In [264]:
report = classification_report(test_y,y_pred1)
print(report)

              precision    recall  f1-score   support

           0       0.93      0.73      0.82        37
           1       0.52      0.85      0.65        13

    accuracy                           0.76        50
   macro avg       0.73      0.79      0.73        50
weighted avg       0.83      0.76      0.77        50



In [265]:
Conf_metrics_sm_rndm = metrics.confusion_matrix(test_y, y_pred1,labels=[0,1])
print(Conf_metrics_sm_rndm)

[[27 10]
 [ 2 11]]


# DECISON TREE

In [266]:
from sklearn.tree import DecisionTreeClassifier
dt_cls = DecisionTreeClassifier(random_state=101)
dt_cls.fit(sm_xtrain,sm_ytrain)
y_pred2=dt_cls.predict(X_test)                                           

In [267]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred2)

0.8284823284823284

In [268]:
report = classification_report(test_y,y_pred2)
print(report)

              precision    recall  f1-score   support

           0       0.94      0.81      0.87        37
           1       0.61      0.85      0.71        13

    accuracy                           0.82        50
   macro avg       0.77      0.83      0.79        50
weighted avg       0.85      0.82      0.83        50



In [269]:
Conf_metrics_sm_rndm = metrics.confusion_matrix(test_y, y_pred2,labels=[0,1])
print(Conf_metrics_sm_rndm)

[[30  7]
 [ 2 11]]


In [270]:
from catboost import CatBoostClassifier
CatBoost=CatBoostClassifier()                                                                                                   
CatBoost.fit(sm_xtrain,sm_ytrain)

Learning rate set to 0.0502
0:	learn: 0.6817153	total: 19.3ms	remaining: 19.3s
1:	learn: 0.6722060	total: 36.9ms	remaining: 18.4s
2:	learn: 0.6637139	total: 53.5ms	remaining: 17.8s
3:	learn: 0.6560889	total: 68ms	remaining: 16.9s
4:	learn: 0.6471249	total: 85.6ms	remaining: 17s
5:	learn: 0.6400910	total: 100ms	remaining: 16.6s
6:	learn: 0.6333071	total: 115ms	remaining: 16.3s
7:	learn: 0.6260021	total: 135ms	remaining: 16.8s
8:	learn: 0.6201837	total: 150ms	remaining: 16.5s
9:	learn: 0.6138735	total: 163ms	remaining: 16.1s
10:	learn: 0.6094077	total: 179ms	remaining: 16.1s
11:	learn: 0.6040042	total: 194ms	remaining: 16s
12:	learn: 0.6001293	total: 210ms	remaining: 15.9s
13:	learn: 0.5958361	total: 224ms	remaining: 15.7s
14:	learn: 0.5910006	total: 238ms	remaining: 15.7s
15:	learn: 0.5860435	total: 255ms	remaining: 15.7s
16:	learn: 0.5836063	total: 272ms	remaining: 15.8s
17:	learn: 0.5807528	total: 288ms	remaining: 15.7s
18:	learn: 0.5765894	total: 305ms	remaining: 15.8s
19:	learn: 0.5

165:	learn: 0.4191122	total: 3.79s	remaining: 19.1s
166:	learn: 0.4189650	total: 3.81s	remaining: 19s
167:	learn: 0.4184574	total: 3.83s	remaining: 19s
168:	learn: 0.4182318	total: 3.85s	remaining: 18.9s
169:	learn: 0.4176404	total: 3.86s	remaining: 18.9s
170:	learn: 0.4174033	total: 3.88s	remaining: 18.8s
171:	learn: 0.4171520	total: 3.9s	remaining: 18.8s
172:	learn: 0.4168944	total: 3.91s	remaining: 18.7s
173:	learn: 0.4166053	total: 3.92s	remaining: 18.6s
174:	learn: 0.4160591	total: 3.94s	remaining: 18.6s
175:	learn: 0.4157497	total: 3.96s	remaining: 18.5s
176:	learn: 0.4152342	total: 3.97s	remaining: 18.5s
177:	learn: 0.4149542	total: 3.99s	remaining: 18.4s
178:	learn: 0.4145879	total: 4.01s	remaining: 18.4s
179:	learn: 0.4138390	total: 4.03s	remaining: 18.3s
180:	learn: 0.4133825	total: 4.04s	remaining: 18.3s
181:	learn: 0.4130028	total: 4.06s	remaining: 18.2s
182:	learn: 0.4124672	total: 4.07s	remaining: 18.2s
183:	learn: 0.4116854	total: 4.09s	remaining: 18.1s
184:	learn: 0.411

328:	learn: 0.3577748	total: 6.5s	remaining: 13.3s
329:	learn: 0.3575388	total: 6.52s	remaining: 13.2s
330:	learn: 0.3572323	total: 6.54s	remaining: 13.2s
331:	learn: 0.3568563	total: 6.56s	remaining: 13.2s
332:	learn: 0.3566667	total: 6.58s	remaining: 13.2s
333:	learn: 0.3562009	total: 6.59s	remaining: 13.1s
334:	learn: 0.3559090	total: 6.61s	remaining: 13.1s
335:	learn: 0.3557217	total: 6.63s	remaining: 13.1s
336:	learn: 0.3555803	total: 6.64s	remaining: 13.1s
337:	learn: 0.3554057	total: 6.66s	remaining: 13s
338:	learn: 0.3551723	total: 6.68s	remaining: 13s
339:	learn: 0.3550099	total: 6.7s	remaining: 13s
340:	learn: 0.3546266	total: 6.72s	remaining: 13s
341:	learn: 0.3544797	total: 6.74s	remaining: 13s
342:	learn: 0.3542133	total: 6.76s	remaining: 12.9s
343:	learn: 0.3538910	total: 6.78s	remaining: 12.9s
344:	learn: 0.3535570	total: 6.8s	remaining: 12.9s
345:	learn: 0.3533175	total: 6.82s	remaining: 12.9s
346:	learn: 0.3531855	total: 6.83s	remaining: 12.9s
347:	learn: 0.3529573	tot

487:	learn: 0.3291072	total: 9.38s	remaining: 9.84s
488:	learn: 0.3289488	total: 9.4s	remaining: 9.82s
489:	learn: 0.3288648	total: 9.42s	remaining: 9.8s
490:	learn: 0.3287762	total: 9.44s	remaining: 9.79s
491:	learn: 0.3285806	total: 9.46s	remaining: 9.77s
492:	learn: 0.3280367	total: 9.48s	remaining: 9.75s
493:	learn: 0.3279306	total: 9.5s	remaining: 9.73s
494:	learn: 0.3275561	total: 9.52s	remaining: 9.71s
495:	learn: 0.3274692	total: 9.53s	remaining: 9.69s
496:	learn: 0.3273461	total: 9.55s	remaining: 9.66s
497:	learn: 0.3272264	total: 9.57s	remaining: 9.64s
498:	learn: 0.3269076	total: 9.58s	remaining: 9.62s
499:	learn: 0.3268314	total: 9.61s	remaining: 9.61s
500:	learn: 0.3266510	total: 9.62s	remaining: 9.59s
501:	learn: 0.3265392	total: 9.64s	remaining: 9.56s
502:	learn: 0.3263995	total: 9.66s	remaining: 9.54s
503:	learn: 0.3262519	total: 9.67s	remaining: 9.52s
504:	learn: 0.3259682	total: 9.69s	remaining: 9.5s
505:	learn: 0.3259146	total: 9.71s	remaining: 9.48s
506:	learn: 0.32

654:	learn: 0.3073887	total: 12.4s	remaining: 6.55s
655:	learn: 0.3072942	total: 12.5s	remaining: 6.54s
656:	learn: 0.3072139	total: 12.5s	remaining: 6.52s
657:	learn: 0.3070261	total: 12.5s	remaining: 6.5s
658:	learn: 0.3068313	total: 12.5s	remaining: 6.48s
659:	learn: 0.3067051	total: 12.6s	remaining: 6.47s
660:	learn: 0.3066183	total: 12.6s	remaining: 6.45s
661:	learn: 0.3065313	total: 12.6s	remaining: 6.43s
662:	learn: 0.3064454	total: 12.6s	remaining: 6.41s
663:	learn: 0.3063457	total: 12.6s	remaining: 6.39s
664:	learn: 0.3062515	total: 12.7s	remaining: 6.38s
665:	learn: 0.3061482	total: 12.7s	remaining: 6.36s
666:	learn: 0.3060301	total: 12.7s	remaining: 6.34s
667:	learn: 0.3059066	total: 12.7s	remaining: 6.32s
668:	learn: 0.3058023	total: 12.7s	remaining: 6.3s
669:	learn: 0.3057121	total: 12.7s	remaining: 6.28s
670:	learn: 0.3056428	total: 12.8s	remaining: 6.26s
671:	learn: 0.3055902	total: 12.8s	remaining: 6.25s
672:	learn: 0.3055011	total: 12.8s	remaining: 6.23s
673:	learn: 0.

816:	learn: 0.2921255	total: 15.6s	remaining: 3.49s
817:	learn: 0.2919788	total: 15.6s	remaining: 3.47s
818:	learn: 0.2918523	total: 15.6s	remaining: 3.45s
819:	learn: 0.2917675	total: 15.6s	remaining: 3.43s
820:	learn: 0.2917076	total: 15.6s	remaining: 3.41s
821:	learn: 0.2916601	total: 15.7s	remaining: 3.39s
822:	learn: 0.2915769	total: 15.7s	remaining: 3.37s
823:	learn: 0.2914876	total: 15.7s	remaining: 3.35s
824:	learn: 0.2913656	total: 15.7s	remaining: 3.33s
825:	learn: 0.2913064	total: 15.7s	remaining: 3.31s
826:	learn: 0.2912186	total: 15.8s	remaining: 3.29s
827:	learn: 0.2911392	total: 15.8s	remaining: 3.28s
828:	learn: 0.2910580	total: 15.8s	remaining: 3.26s
829:	learn: 0.2909852	total: 15.8s	remaining: 3.24s
830:	learn: 0.2909347	total: 15.8s	remaining: 3.22s
831:	learn: 0.2908419	total: 15.8s	remaining: 3.2s
832:	learn: 0.2907668	total: 15.9s	remaining: 3.18s
833:	learn: 0.2906996	total: 15.9s	remaining: 3.16s
834:	learn: 0.2906449	total: 15.9s	remaining: 3.14s
835:	learn: 0

979:	learn: 0.2792342	total: 18.5s	remaining: 377ms
980:	learn: 0.2791897	total: 18.5s	remaining: 358ms
981:	learn: 0.2791212	total: 18.5s	remaining: 339ms
982:	learn: 0.2790547	total: 18.5s	remaining: 320ms
983:	learn: 0.2789933	total: 18.5s	remaining: 302ms
984:	learn: 0.2789218	total: 18.6s	remaining: 283ms
985:	learn: 0.2788467	total: 18.6s	remaining: 264ms
986:	learn: 0.2787747	total: 18.6s	remaining: 245ms
987:	learn: 0.2786653	total: 18.6s	remaining: 226ms
988:	learn: 0.2785281	total: 18.6s	remaining: 207ms
989:	learn: 0.2784293	total: 18.6s	remaining: 188ms
990:	learn: 0.2783656	total: 18.7s	remaining: 169ms
991:	learn: 0.2783015	total: 18.7s	remaining: 151ms
992:	learn: 0.2782556	total: 18.7s	remaining: 132ms
993:	learn: 0.2782158	total: 18.7s	remaining: 113ms
994:	learn: 0.2781078	total: 18.7s	remaining: 94.1ms
995:	learn: 0.2780545	total: 18.7s	remaining: 75.2ms
996:	learn: 0.2779839	total: 18.8s	remaining: 56.4ms
997:	learn: 0.2779161	total: 18.8s	remaining: 37.6ms
998:	lea

In [271]:
y_pred3=CatBoost.predict(np.array(X_test))

In [272]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, y_pred3)

0.6361746361746361

In [273]:
Conf_metrics_sm_cat = metrics.confusion_matrix(test_y, y_pred3,labels=[0,1])
print(Conf_metrics_sm_cat)

[[30  7]
 [ 7  6]]


In [280]:
from sklearn.ensemble import VotingClassifier
model = VotingClassifier(estimators=[('rf', RF_model),('dt', dt_cls)], voting='soft',weights=[1,2])
model.fit(X_train,y_train)

print('Model Complete')
# In[55]:


Ensemble_pred=model.predict(X_test)

Learning rate set to 0.054522
0:	learn: 0.6752092	total: 13.7ms	remaining: 13.7s
1:	learn: 0.6599264	total: 27ms	remaining: 13.5s
2:	learn: 0.6464621	total: 41.7ms	remaining: 13.9s
3:	learn: 0.6338050	total: 56.4ms	remaining: 14s
4:	learn: 0.6225083	total: 70.7ms	remaining: 14.1s
5:	learn: 0.6112846	total: 85.9ms	remaining: 14.2s
6:	learn: 0.6037331	total: 102ms	remaining: 14.5s
7:	learn: 0.5960821	total: 120ms	remaining: 14.8s
8:	learn: 0.5897333	total: 135ms	remaining: 14.9s
9:	learn: 0.5830221	total: 156ms	remaining: 15.5s
10:	learn: 0.5777605	total: 171ms	remaining: 15.4s
11:	learn: 0.5736843	total: 188ms	remaining: 15.5s
12:	learn: 0.5693108	total: 205ms	remaining: 15.6s
13:	learn: 0.5653112	total: 224ms	remaining: 15.8s
14:	learn: 0.5620833	total: 241ms	remaining: 15.8s
15:	learn: 0.5595661	total: 283ms	remaining: 17.4s
16:	learn: 0.5571249	total: 302ms	remaining: 17.5s
17:	learn: 0.5548477	total: 317ms	remaining: 17.3s
18:	learn: 0.5526908	total: 334ms	remaining: 17.3s
19:	learn

163:	learn: 0.5182175	total: 4.26s	remaining: 21.7s
164:	learn: 0.5181472	total: 4.29s	remaining: 21.7s
165:	learn: 0.5180976	total: 4.31s	remaining: 21.6s
166:	learn: 0.5180095	total: 4.33s	remaining: 21.6s
167:	learn: 0.5179457	total: 4.36s	remaining: 21.6s
168:	learn: 0.5178532	total: 4.38s	remaining: 21.6s
169:	learn: 0.5177691	total: 4.41s	remaining: 21.5s
170:	learn: 0.5177025	total: 4.43s	remaining: 21.5s
171:	learn: 0.5176162	total: 4.45s	remaining: 21.4s
172:	learn: 0.5175332	total: 4.47s	remaining: 21.4s
173:	learn: 0.5174438	total: 4.49s	remaining: 21.3s
174:	learn: 0.5174064	total: 4.52s	remaining: 21.3s
175:	learn: 0.5173407	total: 4.54s	remaining: 21.3s
176:	learn: 0.5172752	total: 4.56s	remaining: 21.2s
177:	learn: 0.5172105	total: 4.58s	remaining: 21.2s
178:	learn: 0.5171293	total: 4.6s	remaining: 21.1s
179:	learn: 0.5170555	total: 4.61s	remaining: 21s
180:	learn: 0.5170172	total: 4.63s	remaining: 21s
181:	learn: 0.5169461	total: 4.65s	remaining: 20.9s
182:	learn: 0.516

326:	learn: 0.5060503	total: 7.4s	remaining: 15.2s
327:	learn: 0.5060053	total: 7.42s	remaining: 15.2s
328:	learn: 0.5059483	total: 7.44s	remaining: 15.2s
329:	learn: 0.5058842	total: 7.46s	remaining: 15.2s
330:	learn: 0.5058436	total: 7.49s	remaining: 15.1s
331:	learn: 0.5057908	total: 7.51s	remaining: 15.1s
332:	learn: 0.5057066	total: 7.54s	remaining: 15.1s
333:	learn: 0.5056258	total: 7.56s	remaining: 15.1s
334:	learn: 0.5055626	total: 7.58s	remaining: 15s
335:	learn: 0.5055061	total: 7.6s	remaining: 15s
336:	learn: 0.5054461	total: 7.62s	remaining: 15s
337:	learn: 0.5053995	total: 7.64s	remaining: 15s
338:	learn: 0.5053535	total: 7.66s	remaining: 14.9s
339:	learn: 0.5052914	total: 7.68s	remaining: 14.9s
340:	learn: 0.5052361	total: 7.71s	remaining: 14.9s
341:	learn: 0.5051822	total: 7.73s	remaining: 14.9s
342:	learn: 0.5051334	total: 7.74s	remaining: 14.8s
343:	learn: 0.5050846	total: 7.76s	remaining: 14.8s
344:	learn: 0.5050165	total: 7.78s	remaining: 14.8s
345:	learn: 0.5049503	

488:	learn: 0.4964438	total: 10.7s	remaining: 11.2s
489:	learn: 0.4963731	total: 10.7s	remaining: 11.1s
490:	learn: 0.4963203	total: 10.7s	remaining: 11.1s
491:	learn: 0.4962541	total: 10.7s	remaining: 11.1s
492:	learn: 0.4962146	total: 10.8s	remaining: 11.1s
493:	learn: 0.4961671	total: 10.8s	remaining: 11s
494:	learn: 0.4961199	total: 10.8s	remaining: 11.1s
495:	learn: 0.4960799	total: 10.9s	remaining: 11.1s
496:	learn: 0.4960133	total: 10.9s	remaining: 11.1s
497:	learn: 0.4959665	total: 11s	remaining: 11.1s
498:	learn: 0.4959009	total: 11s	remaining: 11s
499:	learn: 0.4958070	total: 11s	remaining: 11s
500:	learn: 0.4957499	total: 11.1s	remaining: 11s
501:	learn: 0.4957020	total: 11.1s	remaining: 11s
502:	learn: 0.4956455	total: 11.1s	remaining: 11s
503:	learn: 0.4956032	total: 11.2s	remaining: 11s
504:	learn: 0.4955638	total: 11.2s	remaining: 11s
505:	learn: 0.4955209	total: 11.2s	remaining: 11s
506:	learn: 0.4954703	total: 11.3s	remaining: 11s
507:	learn: 0.4954079	total: 11.3s	rem

649:	learn: 0.4881707	total: 15.8s	remaining: 8.53s
650:	learn: 0.4881220	total: 15.9s	remaining: 8.51s
651:	learn: 0.4880834	total: 15.9s	remaining: 8.49s
652:	learn: 0.4880214	total: 15.9s	remaining: 8.47s
653:	learn: 0.4879676	total: 16s	remaining: 8.44s
654:	learn: 0.4879160	total: 16s	remaining: 8.42s
655:	learn: 0.4878768	total: 16s	remaining: 8.4s
656:	learn: 0.4878226	total: 16s	remaining: 8.37s
657:	learn: 0.4877837	total: 16.1s	remaining: 8.35s
658:	learn: 0.4877279	total: 16.1s	remaining: 8.32s
659:	learn: 0.4876801	total: 16.1s	remaining: 8.3s
660:	learn: 0.4876423	total: 16.1s	remaining: 8.27s
661:	learn: 0.4876073	total: 16.2s	remaining: 8.25s
662:	learn: 0.4875774	total: 16.2s	remaining: 8.22s
663:	learn: 0.4875148	total: 16.2s	remaining: 8.2s
664:	learn: 0.4874667	total: 16.2s	remaining: 8.17s
665:	learn: 0.4874166	total: 16.2s	remaining: 8.14s
666:	learn: 0.4873653	total: 16.3s	remaining: 8.12s
667:	learn: 0.4873211	total: 16.3s	remaining: 8.09s
668:	learn: 0.4872734	t

810:	learn: 0.4805794	total: 20.2s	remaining: 4.71s
811:	learn: 0.4805361	total: 20.3s	remaining: 4.69s
812:	learn: 0.4804991	total: 20.3s	remaining: 4.66s
813:	learn: 0.4804677	total: 20.3s	remaining: 4.64s
814:	learn: 0.4804356	total: 20.3s	remaining: 4.61s
815:	learn: 0.4803962	total: 20.3s	remaining: 4.58s
816:	learn: 0.4803166	total: 20.4s	remaining: 4.56s
817:	learn: 0.4802567	total: 20.4s	remaining: 4.53s
818:	learn: 0.4802101	total: 20.4s	remaining: 4.51s
819:	learn: 0.4801449	total: 20.4s	remaining: 4.48s
820:	learn: 0.4801223	total: 20.4s	remaining: 4.46s
821:	learn: 0.4800688	total: 20.5s	remaining: 4.43s
822:	learn: 0.4800422	total: 20.5s	remaining: 4.41s
823:	learn: 0.4799923	total: 20.5s	remaining: 4.38s
824:	learn: 0.4799543	total: 20.5s	remaining: 4.36s
825:	learn: 0.4799072	total: 20.6s	remaining: 4.33s
826:	learn: 0.4798620	total: 20.6s	remaining: 4.31s
827:	learn: 0.4798181	total: 20.6s	remaining: 4.29s
828:	learn: 0.4797747	total: 20.7s	remaining: 4.26s
829:	learn: 

971:	learn: 0.4735787	total: 23.8s	remaining: 685ms
972:	learn: 0.4735557	total: 23.8s	remaining: 661ms
973:	learn: 0.4735037	total: 23.9s	remaining: 637ms
974:	learn: 0.4734472	total: 23.9s	remaining: 612ms
975:	learn: 0.4734210	total: 23.9s	remaining: 588ms
976:	learn: 0.4733713	total: 23.9s	remaining: 563ms
977:	learn: 0.4733455	total: 23.9s	remaining: 539ms
978:	learn: 0.4733132	total: 24s	remaining: 514ms
979:	learn: 0.4732733	total: 24s	remaining: 490ms
980:	learn: 0.4732112	total: 24s	remaining: 465ms
981:	learn: 0.4731698	total: 24s	remaining: 441ms
982:	learn: 0.4731195	total: 24.1s	remaining: 416ms
983:	learn: 0.4730836	total: 24.1s	remaining: 392ms
984:	learn: 0.4730436	total: 24.1s	remaining: 367ms
985:	learn: 0.4730127	total: 24.1s	remaining: 343ms
986:	learn: 0.4729732	total: 24.1s	remaining: 318ms
987:	learn: 0.4729270	total: 24.2s	remaining: 294ms
988:	learn: 0.4728947	total: 24.2s	remaining: 269ms
989:	learn: 0.4728588	total: 24.2s	remaining: 245ms
990:	learn: 0.472799

In [281]:
from sklearn.metrics import roc_auc_score
roc_auc_score(test_y, Ensemble_pred)

0.5592515592515592

In [282]:
Conf_metrics_emsem = metrics.confusion_matrix(test_y, Ensemble_pred,labels=[0,1])
print(Conf_metrics_emsem)

[[30  7]
 [ 9  4]]


In [276]:
Conf_metrics_emsem = metrics.confusion_matrix(test_y, Ensemble_pred,labels=[0,1])
print(Conf_metrics_emsem)

[[30  7]
 [ 9  4]]


# VALIDATION

In [283]:
X1=d1.loc[:, d1.columns != 'ID']
y1=d1['Response']
X1.drop(columns=['Response'],inplace=True)
X1=X1[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age','Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat','Reco_Policy_Premium','Holding_Policy_Type','Reco_Insurance_Type','Is_Spouse']]

C:\Python\lib\site-packages\pandas\core\frame.py:4117: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [284]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X,y, test_size=0.20, random_state=100)

In [165]:
from numpy import mean
from numpy import std

In [ ]:
# create dataset
# X, y = make_classification(n_samples=100, n_features=20, n_informative=15, n_redundant=5, random_state=1)
# prepare the cross-validation procedure
cv = KFold(n_splits=10, random_state=1, shuffle=True)
# create model
model = model
# evaluate model
scores = cross_val_score(model, X1_train, y1_train, scoring="roc_auc", cv=cv, n_jobs=-1)
# report performance
print('roc_auc: %.3f (%.3f)' % (mean(scores), std(scores)))

In [237]:
X_train.columns

Index(['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age',
       'Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat',
       'Reco_Policy_Premium', 'Holding_Policy_Type', 'Reco_Insurance_Type',
       'Is_Spouse'],
      dtype='object')

# TEST

In [224]:
Test=pd.read_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\test_YCcRUnU.csv')

In [225]:
Test.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium
0,50883,C1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0
1,50884,C4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8
2,50885,C1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0
3,50886,C3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0
4,50887,C1,951,Owned,Individual,75,75,No,X3,NaN,NaN,5,22534.0


In [226]:
Test['Holding_Policy_Duration']=Test['Holding_Policy_Duration'].replace('14+',15)

In [227]:
Test=Test.astype({"Holding_Policy_Type":'category', "Reco_Policy_Cat":'category',"City_Code":'category',"Accomodation_Type":'category',"Reco_Insurance_Type":'category',"Holding_Policy_Duration":'category',"Is_Spouse":'category',"Health Indicator":'category','Region_Code':'category'})

In [228]:
Test['Accomodation_Type']=Test['Accomodation_Type'].cat.codes
Test['Reco_Insurance_Type']=Test['Reco_Insurance_Type'].cat.codes
Test['City_Code']=Test['City_Code'].cat.codes

In [229]:
Test['Health Indicator']=Test['Health Indicator'].cat.codes
Test['Health Indicator']=Test['Health Indicator'].replace(-1, np.nan)

In [230]:
Test.columns

Index(['ID', 'City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium'],
      dtype='object')

In [ ]:
'City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age',
       'Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat',
       'Reco_Policy_Premium', 'Holding_Policy_Type', 'Reco_Insurance_Type',
       'Is_Spouse'

In [231]:
Test=Test[['ID','City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age','Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat','Reco_Policy_Premium','Holding_Policy_Type','Accomodation_Type','Reco_Insurance_Type']]

In [232]:
from sklearn.impute import SimpleImputer
my_imputer = SimpleImputer()
Test_data = pd.DataFrame(my_imputer.fit_transform(Test))
Test_data.columns=Test.columns
Test_data.index=Test.index

In [233]:
Test_data.columns

Index(['ID', 'City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age',
       'Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat',
       'Reco_Policy_Premium', 'Holding_Policy_Type', 'Accomodation_Type',
       'Reco_Insurance_Type'],
      dtype='object')

In [234]:
Test_data[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']]=StandardScaler().fit_transform(Test_data[['City_Code','Region_Code', 'Upper_Age', 'Lower_Age', 'Health Indicator','Holding_Policy_Duration', 'Holding_Policy_Type', 'Reco_Policy_Cat','Reco_Policy_Premium']])

In [235]:
Final_Test_data=Test_data.drop(columns=['ID'])

In [238]:
Final_Test_data[['City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age','Health Indicator', 'Holding_Policy_Duration', 'Reco_Policy_Cat','Reco_Policy_Premium', 'Holding_Policy_Type', 'Reco_Insurance_Type','Is_Spouse']]

KeyError: "['Is_Spouse'] not in index"

In [236]:
y_final=model.predict_proba(Final_Test_data)[:,1]

CatBoostError: c:/program files (x86)/go agent/pipelines/buildmaster/catboost.git/catboost/libs/data/model_dataset_compatibility.cpp:81: At position 9 should be feature with name Reco_Insurance_Type (found Accomodation_Type).

In [ ]:
y_final

In [ ]:
y_final

In [ ]:
Test_data['Response']=y_final

In [ ]:
Test_data.columns

In [ ]:
result=Test_data[['ID','Response']]

In [ ]:
result['Response'].value_counts()

In [ ]:
result['Response'].value_counts()

In [ ]:
result['Response'].value_counts()

In [ ]:
result['Response'].value_counts()

In [ ]:
result.head()

In [ ]:
result.to_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\dt.csv')

In [ ]:
check=pd.read_csv(r'C:\Users\aditya_jyoti\Documents\Docs\Hackathon\Random_forest.csv')

In [ ]:
predicted=[0 if i > 0.70 else 1 for i in y_prob1]

In [ ]:
check['Response'].apply(lambda x:1 if x > 0.5 else 0 ).value_counts()

In [ ]:
X_train.columns

In [ ]:
df.columns

In [ ]:
df['Reco_Insurance_Type'].value_counts()

In [ ]:
df['Is_Spouse'].value_counts()